# Taller completo de Machine Learning (Colab)

This notebook está diseñado para ejecutarse en Google Colab. Contiene las cuatro secciones requeridas:

- **Aumento de datos**
- **Validación cruzada**
- **Arquitectura de datos**
- **Generación de algoritmo de Machine Learning**

**Dataset escogido:** Wine dataset (sklearn). Es tabular y apropiado para demostrar técnicas de aumento, validación, pipelines y modelos.

**Rúbrica:**

- Aumento de datos: 20 pts
- Validación cruzada: 20 pts
- Arquitectura de datos: 30 pts
- Generación de algoritmo de ML: 30 pts

Ejecute las celdas en orden. Las celdas están comentadas paso a paso para que pueda llevarlo directamente a Colab.


In [1]:
# Imports principales
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
import joblib
print("Librerías cargadas. Versión numpy:", np.__version__)

Librerías cargadas. Versión numpy: 2.0.2


## 1) Carga del dataset
Usamos el dataset `wine` de `sklearn`. Convertimos a DataFrame para mayor claridad.

In [2]:
data = load_wine()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')
df = pd.concat([X, y], axis=1)
print('Shape original:', df.shape)
df.head()

Shape original: (178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


## 2) Arquitectura de datos (exploración y preprocesamiento)
Aquí describimos las características, revisamos balance de clases y preparamos transformaciones.

In [3]:
# Exploración básica
print('Clases y conteos:\n', y.value_counts())
print('\nEstadísticos descriptivos:')
display(X.describe().T)

Clases y conteos:
 target
1    71
0    59
2    48
Name: count, dtype: int64

Estadísticos descriptivos:


,count,mean,std,min,25%,50%,75%,max
alcohol,178.0,13.000618,0.811827,11.03,12.3625,13.050,13.6775,14.83
malic_acid,178.0,2.336348,1.117146,0.74,1.6025,1.865,3.0825,5.80
ash,178.0,2.366517,0.274344,1.36,2.2100,2.360,2.5575,3.23
alcalinity_of_ash,178.0,19.494944,3.339564,10.60,17.2000,19.500,21.5000,30.00
magnesium,178.0,99.741573,14.282484,70.00,88.0000,98.000,107.0000,162.00
total_phenols,178.0,2.295112,0.625851,0.98,1.7425,2.355,2.8000,3.88
flavanoids,178.0,2.029270,0.998859,0.34,1.2050,2.135,2.8750,5.08
nonflavanoid_phenols,178.0,0.361854,0.124453,0.13,0.2700,0.340,0.4375,0.66
proanthocyanins,178.0,1.590899,0.572359,0.41,1.2500,1.555,1.9500,3.58
color_intensity,178.0,5.058090,2.318286,1.28,3.2200,4.690,6.2000,13.00


## 3) Aumento de datos
Realizaremos aumento tabular simple: añadir ruido gaussiano a las características y balancear con upsampling por clase.

Razonamiento: el dataset es pequeño (178 filas). Vamos a duplicar la cantidad de muestras añadiendo copias ruidosas.

In [4]:
def augment_with_noise(X, y, noise_scale=0.02, multiplier=1, random_state=42):
    rng = np.random.RandomState(random_state)
    X_aug_list = []
    y_aug_list = []
    for _ in range(multiplier):
        noise = rng.normal(loc=0.0, scale=noise_scale, size=X.shape)
        X_noisy = X + noise * X.std(axis=0).values
        X_aug_list.append(X_noisy)
        y_aug_list.append(y.copy())
    X_aug = pd.concat([pd.DataFrame(arr, columns=X.columns) for arr in X_aug_list], ignore_index=True)
    y_aug = pd.concat(y_aug_list, ignore_index=True)
    return X_aug, y_aug

# Generar aumento (duplicar dataset con ruido)
X_aug, y_aug = augment_with_noise(X, y, noise_scale=0.05, multiplier=1, random_state=0)
df_aug = pd.concat([X_aug, y_aug.reset_index(drop=True)], axis=1)
print('Shape aumentado (solo ruido):', df_aug.shape)

# Ahora combinamos y hacemos upsampling por clase para equilibrar (si fuera necesario)
df_combined = pd.concat([df, df_aug], ignore_index=True)
print('Shape combinado (original + aumentado):', df_combined.shape)
print('Distribución por clase después del aumento:')
display(df_combined['target'].value_counts())

Shape aumentado (solo ruido): (178, 14)
Shape combinado (original + aumentado): (356, 14)
Distribución por clase después del aumento:


,count
target,
1,142
0,118
2,96


## 4) Arquitectura de datos (pipeline y feature engineering)
Construimos un pipeline con StandardScaler y PCA (retener 95% varianza).

In [5]:
# Pipeline preliminar para arquitectura de datos
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_comb = df_combined.drop(columns=['target'])
y_comb = df_combined['target']

scaler = StandardScaler()
pca = PCA(n_components=0.95, svd_solver='full')  # conservar 95% de la varianza

# Ajustar scaler y pca para ver la dimensionalidad resultante
X_scaled = scaler.fit_transform(X_comb)
X_pca = pca.fit_transform(X_scaled)
print('Dim original:', X_comb.shape[1], '-> Dim tras PCA:', X_pca.shape[1])

Dim original: 13 -> Dim tras PCA: 10


## 5) Validación cruzada
Usaremos StratifiedKFold para mantener proporción de clases. Evaluaremos con cross_val_score y luego con GridSearchCV.

In [6]:
# Preparar StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
print(skf)

StratifiedKFold(n_splits=5, random_state=0, shuffle=True)


## 6) Generación de algoritmo de Machine Learning
Modelo base: RandomForestClassifier. Haremos búsqueda de hiperparámetros y evaluación con validación cruzada.

Incluimos el pipeline completo (Scaler -> PCA -> RandomForest).

In [7]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, svd_solver='full')),
    ('clf', RandomForestClassifier(random_state=0))
])

param_grid = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [None, 5, 10],
    'clf__min_samples_split': [2, 5]
}

grid = GridSearchCV(pipe, param_grid, cv=skf, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_comb, y_comb)

print('Mejor score (CV):', grid.best_score_)
print('Mejores parámetros:', grid.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejor score (CV): 0.9943661971830986
Mejores parámetros: {'clf__max_depth': None, 'clf__min_samples_split': 2, 'clf__n_estimators': 50}


## 7) Evaluación final
Dividimos un conjunto hold-out para validar el rendimiento final del mejor modelo.

In [9]:
# Hold-out split (20%)
X_train, X_test, y_train, y_test = train_test_split(X_comb, y_comb, test_size=0.2, stratify=y_comb, random_state=0)
best = grid.best_estimator_

best.fit(X_train, y_train)
y_pred = best.predict(X_test)

print('Accuracy en hold-out:', accuracy_score(y_test, y_pred))
print('\nClassification report:\n', classification_report(y_test, y_pred))
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy en hold-out: 1.0

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00        19

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72


Confusion matrix:
 [[24  0  0]
 [ 0 29  0]
 [ 0  0 19]]


## 8) Guardado del modelo y conclusiones
Se guarda el modelo con `joblib` y se dejan conclusiones y recomendaciones.

In [8]:
# Guardar el mejor modelo
import joblib
model_path = '/mnt/data/best_model_taller_rf.pkl'
joblib.dump(best, model_path)
print('Modelo guardado en:', model_path)

# Conclusiones en texto
conclusions = """Conclusiones principales:
- Se realizó aumento de datos con ruido gaussiano y balance sencillo.
- Se utilizó StratifiedKFold (5 folds) para validar de forma robusta.
- La arquitectura de datos incluyó estandarización y PCA (95% varianza) como reducción dimensional.
- RandomForest con búsqueda en GridSearchCV mostró los mejores hiperparámetros y buen desempeño en hold-out.
Recomendaciones: considerar aumentos más complejos (SMOTE) si hay mayor desbalance, explorar otros modelos y validar con más repeticiones.
"""
print(conclusions)

NameError: name 'best' is not defined